__качаем объявления о продаже земельных участков с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/zemelnye_uchastki/prodam/izhs'
    
raw_data_path = 'data/raw/land/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-11-07 10:45:10 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-07 10:45:10 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-07 10:45:13 | AdsListParser: downloader init
[INFO    ] 2022-11-07 10:45:13 | AvitoParser: init
[INFO    ] 2022-11-07 10:45:20 | AvitoParser: 21 pages for read
[INFO    ] 2022-11-07 10:45:20 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-07 10:45:22 | AdsListParser: read page 1
[INFO    ] 2022-11-07 10:45:23 | AdsListParser: read page 2
[INFO    ] 2022-11-07 10:45:25 | AdsListParser: read page 3
[INFO    ] 2022-11-07 10:45:30 | AdsListParser: read page 4
[INFO    ] 2022-11-07 10:45:50 | AdsListParser: read page 5
[INFO    ] 2022-11-07 10:45:52 | AdsListParser: read page 6
[INFO    ] 2022-11-07 10:45:54 | AdsListParser: read page 7
[INFO    ] 2022-11-07 10:45:58 | AdsListParser: read page 8
[INFO    ] 2022-11-07 10:46:01 | AdsListParser: read page 9
[INFO    ] 2022-11-07 10:46:03 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

1036
1028


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
649,2528227909,"Объявление «Участок 3,6 сот. (ИЖС)» 7 фотографий",3150000,,"Севастополь, улица Штурмана Батурина, 3","Участок 3.61 сотки Ижс. \n\nУчасток 3,61 сотки, расположен в г. Севастополь, Гагаринский район, ул. Штурмана Батурина д 1. Участок ровный. Без уклона. \n\nПозвоните нам, чтобы узнать все детали! \n\nКоммуникации: \n\n· Электричество — 15 квт, 3 фазы, 380 V.\n\n· Вод",14,2022-11-07 10:46:32.362559,sevastopol
227,2464833158,Объявление «Участок 5 сот. (ИЖС)» 5 фотографий,1400000,,"с. Фруктовое, Виноградная ул., 4А","Предлагаю вашему вниманию хороший участок в Нахимовском районе, с. Фруктовое. \n\nУчасток общей площадью Ижс 5 сотки прямоугольной формы.\n\nКоммуникации: \n\n- вода городская.\n\n- свет трёхфазный городской (есть ТУ).\n\n- газ в перспективе.\n\n- Оформлен Гпзу.\n\nПреи",5,2022-11-07 10:46:32.362559,sevastopol
